# Installing and downloading dataset from Roboflow

In [1]:
!pip install roboflow

## Using Roboflow to get dataset

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="FnqyiwQGPFDUNJqVepi7")
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(1)
dataset = version.download("yolov9")


loading Roboflow workspace...
loading Roboflow project...


Now that we have downloaded the dataset, We are going to train the model for different classes.

The initial problems that we encountered so far are as follows

1 - Detecting players outside the pitch mainly substitutes
2 - Detecting Coach and managers as persons
3 - Detecting referee as person
4 - Not detecting ball (which is very important for our analysis)

In [3]:
dataset.location

'/content/football-players-detection-1'

# Training the Model

In [4]:
!pip install ultralytics

In [23]:
import os

dataset_path = "/content/football-players-detection-1/football-players-detection-1" # Corrected path

# List the contents of the dataset directory
print(os.listdir(dataset_path))

# List the contents of the images and labels directories within the nested directory
print(os.listdir(os.path.join(dataset_path, 'images')))
print(os.listdir(os.path.join(dataset_path, 'labels')))

['images', 'test', 'labels', 'valid']
['744b27_3_3_png.rf.730f04ca0c8bd35dd75a3832fbeb438d.jpg', '4b770a_3_2_png.rf.494474c95db58f028f83ea8e27ae72b3.jpg', '42ba34_1_6_png.rf.62f6b27513b5d6b4a52135e201326b3c.jpg', '538438_7_3_png.rf.9f25835533a61559749ced615dd61b9c.jpg', '573e61_5_9_png.rf.18f3a071f36eabded0faa69edf092834.jpg', '2e57b9_1_8_png.rf.eddb5d52cda524e90e21bc091077ef2d.jpg', '744b27_5_1_png.rf.91c9243585edd4a7b93c4624dc5cc171.jpg', '744b27_7_2_png.rf.a11bea85f60a8c881adfde04c19efd7c.jpg', '2e57b9_9_3_png.rf.e21e0b84e22c9c2c2f4a5795297980b7.jpg', '40cd38_3_3_png.rf.f0ed10e6b707cf2bc141cba84660726f.jpg', '744b27_3_1_png.rf.29540e286d40f0fcd4b16beb145543bc.jpg', '121364_9_5_png.rf.ac64b2f34de8deb447a4720f4014f4a6.jpg', '798b45_5_4_png.rf.32bbee32eef2242fa9701b331fcc34fc.jpg', '744b27_1_5_png.rf.c36c189f1922007f17a5e84c3c78966e.jpg', '08fd33_6_6_png.rf.ca1d8cba1caabb2151711cc48552a923.jpg', 'cd987c_0_2_png.rf.6e5034f2fba18d9a5d1b5e8a964ac97f.jpg', '42ba34_9_9_png.rf.9aeac5259d259b

In [5]:
from ultralytics import YOLO

"""# Build a YOLOv9c model from scratch
model = YOLO("yolov9c.yaml")
"""
# Build a YOLOv9c model from pretrained weight
model = YOLO("yolov9t.pt")

# Display model information (optional)
model.info()

# Modify the data.yaml file to fix the paths
import yaml

data_yaml_path = "/content/football-players-detection-1/data.yaml"

with open(data_yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)

# Update the paths based on the confirmed directory structure
data_yaml['train'] = '/content/football-players-detection-1/football-players-detection-1/images'
data_yaml['val'] = '/content/football-players-detection-1/football-players-detection-1/images'
data_yaml['test'] = '/content/football-players-detection-1/football-players-detection-1/images'

# Update label paths as well
data_yaml['train_txt'] = '/content/football-players-detection-1/football-players-detection-1/labels'
data_yaml['val_txt'] = '/content/football-players-detection-1/football-players-detection-1/labels'
data_yaml['test_txt'] = '/content/football-players-detection-1/football-players-detection-1/labels'


# Print the updated paths to verify
print("Updated data.yaml paths:")
print(f"train: {data_yaml['train']}")
print(f"val: {data_yaml['val']}")
print(f"test: {data_yaml['test']}")
print(f"train_txt: {data_yaml['train_txt']}")
print(f"val_txt: {data_yaml['val_txt']}")
print(f"test_txt: {data_yaml['test_txt']}")


with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data=data_yaml_path, epochs=100, imgsz=640, device=0, name="football-yolov9c100epocs")

YOLOv9t summary: 544 layers, 2,128,720 parameters, 0 gradients, 8.5 GFLOPs
Updated data.yaml paths:
train: /content/football-players-detection-1/football-players-detection-1/images
val: /content/football-players-detection-1/football-players-detection-1/images
test: /content/football-players-detection-1/football-players-detection-1/images
train_txt: /content/football-players-detection-1/football-players-detection-1/labels
val_txt: /content/football-players-detection-1/football-players-detection-1/labels
test_txt: /content/football-players-detection-1/football-players-detection-1/labels
Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/football-players-detection-1/data.yaml, degrees=0.0, deter

In [4]:
import os
import zipfile
from google.colab import files

weights_dir = "/content/runs/detect/football-yolov9c100epocs"
output_filename = "final-nodel-yolov9c.zip"

# Zip the weights directory
with zipfile.ZipFile(output_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, filenames in os.walk(weights_dir): # Renamed 'files' to 'filenames'
        for filename in filenames: # Renamed 'file' to 'filename'
            zipf.write(os.path.join(root, filename), os.path.relpath(os.path.join(root, filename), weights_dir))

# Download the zip file
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>